# [1] 주최측 temp에 우리가 만든 column merge
20200720 신혜연   
아는 걸로,,,, 업로드 두 버전으로 해봤어요! 둘 다 틀렸을 수도 있습니다.. 한 번 읽어봐주세요  
temp + df_num / temp + train_num  
읽어보시고 (만약에 둘 다 오류가 없다면) 뭘로 할지 월요일( 오늘 ) 결정하면 될 듯합니다.  
따로 temp를 npy파일 형태로 저장할까 고민했지만, 돌리는데 시간이 그리 걸리지 않아 따로 저장은 하지 않았습니다.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('C:/Users/USER/Desktop/2020 summer dacon/2020_jeju_credit card/code/modeling/modeling_data.csv', encoding = 'CP949')

In [3]:
# feature, target 설정
df_num = data

In [4]:
df_num

,Unnamed: 0,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,month,year,deceased,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby
0,1,강원,건강보조식품 소매업,강원,20s,1,1,843100.0,11,2019,0,0,8.806154,False,1.562276e+13,42767.077,10.591,1.980289e+06
1,2,강원,건강보조식품 소매업,강원,20s,1,1,139000.0,6,2019,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
2,3,강원,건강보조식품 소매업,강원,20s,1,1,27500.0,8,2019,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
3,4,강원,건강보조식품 소매업,강원,20s,1,1,395500.0,9,2019,0,11,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
4,5,강원,건강보조식품 소매업,강원,20s,1,1,427510.0,3,2020,1,0,7.066787,True,1.562276e+13,42767.077,10.591,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057389,1057390,충북,휴양콘도 운영업,충북,70s,2,5,230400.0,1,2020,0,0,7.066787,False,4.267602e+11,42767.080,10.591,NaN
1057390,1057391,충북,휴양콘도 운영업,충북,70s,2,5,247800.0,12,2019,0,0,8.806154,False,4.267602e+11,42767.080,10.591,1.433245e+05
1057391,1057392,충북,휴양콘도 운영업,충북,70s,2,5,83000.0,2,2019,0,0,7.066787,False,4.267602e+11,42767.080,10.591,1.433245e+05
1057392,1057393,충북,휴양콘도 운영업,충북,70s,2,5,329800.0,5,2019,0,0,7.066787,False,4.267602e+11,42767.080,10.591,1.433245e+05


In [5]:
# feature, target 설정
train_num = df_num.sample(frac=1, random_state=0)
train_features = train_num.drop(['AMT'], axis=1)
train_target = np.log1p(train_num['AMT'])

In [6]:
# 예측 템플릿 만들기
CARD_SIDO_NMs = df_num['CARD_SIDO_NM'].unique()
STD_CLSS_NMs  = df_num['STD_CLSS_NM'].unique()
HOM_SIDO_NMs  = df_num['HOM_SIDO_NM'].unique()
AGEs          = df_num['AGE'].unique()
SEX_CTGO_CDs  = df_num['SEX_CTGO_CD'].unique()
FLCs          = df_num['FLC'].unique()
years         = [2020]
months        = [4, 7]

temp = []
for CARD_SIDO_NM in CARD_SIDO_NMs:
    for STD_CLSS_NM in STD_CLSS_NMs:
        for HOM_SIDO_NM in HOM_SIDO_NMs:
            for AGE in AGEs:
                for SEX_CTGO_CD in SEX_CTGO_CDs:
                    for FLC in FLCs:
                        for month in months:
                            for year in years:
                                temp.append([CARD_SIDO_NM, STD_CLSS_NM, HOM_SIDO_NM, AGE, SEX_CTGO_CD, FLC, month, year])

temp = np.array(temp)

In [7]:
our_train_features_col = train_features.columns.to_list()
print(our_train_features_col, "total length " + str(len(our_train_features_col)))

['Unnamed: 0', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC', 'month', 'year', 'deceased', 'festival_count', 'rain_day', 'covid', 'age_population', 'total_sale_by_indst_groupby', 'log_total_sale_by_indst', 'total_tourist_groupby'] total length 17


In [8]:
origin_train_features_col = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC' , 'year', 'month'] 

In [9]:
# 대회 측에 맞게 columns변경
temp = pd.DataFrame(data=temp, columns=origin_train_features_col)

In [10]:
temp.shape

(1658860, 8)

In [11]:
temp

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month
0,강원,건강보조식품 소매업,강원,20s,1,1,4,2020
1,강원,건강보조식품 소매업,강원,20s,1,1,7,2020
2,강원,건강보조식품 소매업,강원,20s,1,2,4,2020
3,강원,건강보조식품 소매업,강원,20s,1,2,7,2020
4,강원,건강보조식품 소매업,강원,20s,1,3,4,2020
...,...,...,...,...,...,...,...,...
1658855,충북,정기 항공 운송업,제주,10s,2,3,7,2020
1658856,충북,정기 항공 운송업,제주,10s,2,4,4,2020
1658857,충북,정기 항공 운송업,제주,10s,2,4,7,2020
1658858,충북,정기 항공 운송업,제주,10s,2,5,4,2020


# dacon의 temp merge with **train_num** : temp_train_num.csv로 저장

In [12]:
train_num

,Unnamed: 0,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,AMT,month,year,deceased,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby
149245,149246,경남,그외 기타 분류안된 오락관련 서비스업,경남,30s,1,2,145000.0,3,2019,0,5,8.806154,False,1.078704e+14,42767.080,10.591,3.040269e+05
554956,554957,서울,일반유흥 주점업,인천,20s,1,2,228000.0,12,2019,0,1,8.806154,False,5.395069e+14,721586.000,13.470,9.279775e+06
918516,918517,제주,차량용 주유소 운영업,제주,50s,2,4,795920911.0,5,2019,0,17,11.461490,False,4.596082e+13,721586.000,13.470,1.980289e+06
425414,425415,부산,서양식 음식점업,서울,30s,2,1,4422765.0,9,2019,0,1,11.461490,False,1.924641e+14,251207.500,12.425,9.279775e+06
640949,640950,세종,화장품 및 방향제 소매업,충북,60s,2,5,622000.0,6,2019,0,0,8.806154,False,6.896152e+11,10972.310,9.074,1.433245e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359783,359784,대전,비알콜 음료점업,세종,20s,1,1,4241035.0,5,2019,0,0,7.066787,False,1.676844e+13,251207.500,12.425,3.040269e+05
152315,152316,경남,기타 대형 종합 소매업,인천,40s,2,3,593960.0,1,2019,0,0,5.297286,False,1.078704e+14,1976481.000,14.486,3.040269e+05
963395,963396,충남,수산물 소매업,세종,60s,1,5,4325000.0,1,2020,0,1,7.066787,False,2.446042e+13,251207.500,12.425,NaN
117952,117953,경기,중식 음식점업,광주,20s,1,2,107440.0,7,2019,0,3,11.461487,False,2.856824e+14,1253926.239,14.035,9.279775e+06


In [13]:
train_num_drop = train_num.drop(['Unnamed: 0','year','month','AMT'],axis = 1)

In [14]:
train_num_drop

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,deceased,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby
149245,경남,그외 기타 분류안된 오락관련 서비스업,경남,30s,1,2,0,5,8.806154,False,1.078704e+14,42767.080,10.591,3.040269e+05
554956,서울,일반유흥 주점업,인천,20s,1,2,0,1,8.806154,False,5.395069e+14,721586.000,13.470,9.279775e+06
918516,제주,차량용 주유소 운영업,제주,50s,2,4,0,17,11.461490,False,4.596082e+13,721586.000,13.470,1.980289e+06
425414,부산,서양식 음식점업,서울,30s,2,1,0,1,11.461490,False,1.924641e+14,251207.500,12.425,9.279775e+06
640949,세종,화장품 및 방향제 소매업,충북,60s,2,5,0,0,8.806154,False,6.896152e+11,10972.310,9.074,1.433245e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
359783,대전,비알콜 음료점업,세종,20s,1,1,0,0,7.066787,False,1.676844e+13,251207.500,12.425,3.040269e+05
152315,경남,기타 대형 종합 소매업,인천,40s,2,3,0,0,5.297286,False,1.078704e+14,1976481.000,14.486,3.040269e+05
963395,충남,수산물 소매업,세종,60s,1,5,0,1,7.066787,False,2.446042e+13,251207.500,12.425,NaN
117952,경기,중식 음식점업,광주,20s,1,2,0,3,11.461487,False,2.856824e+14,1253926.239,14.035,9.279775e+06


In [15]:
type(train_num)

pandas.core.frame.DataFrame

In [16]:
# ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat
train_num_drop.dtypes

CARD_SIDO_NM                    object
STD_CLSS_NM                     object
HOM_SIDO_NM                     object
AGE                             object
SEX_CTGO_CD                      int64
FLC                              int64
deceased                         int64
festival_count                   int64
rain_day                       float64
covid                             bool
age_population                 float64
total_sale_by_indst_groupby    float64
log_total_sale_by_indst        float64
total_tourist_groupby          float64
dtype: object

In [17]:
# ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat
temp.dtypes
int_features = ['SEX_CTGO_CD','FLC']
for i in enumerate (int_features) : 
    ca = i[1] 
    temp[ca] = temp[ca].astype('int64') 

In [18]:
temp_train_num = pd.merge(temp, train_num_drop, on=['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC'])

In [19]:
temp_train_num.shape

(2114788, 16)

In [20]:
temp_train_num.columns

Index(['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD',
       'FLC', 'year', 'month', 'deceased', 'festival_count', 'rain_day',
       'covid', 'age_population', 'total_sale_by_indst_groupby',
       'log_total_sale_by_indst', 'total_tourist_groupby'],
      dtype='object')

In [21]:
temp_train_num

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,deceased,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby
0,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,0,0,5.297286,False,1.562276e+13,42767.077,10.591,NaN
1,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,0,4,5.297286,False,1.562276e+13,42767.077,10.591,1.980289e+06
2,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,0,4,5.297286,False,1.562276e+13,42767.077,10.591,1.980289e+06
3,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,0,0,8.806154,False,1.562276e+13,42767.077,10.591,1.980289e+06
4,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114783,충북,휴양콘도 운영업,제주,50s,1,4,4,2020,0,0,11.461490,False,4.741606e+13,42767.080,10.591,NaN
2114784,충북,휴양콘도 운영업,제주,50s,1,4,4,2020,0,2,8.806154,False,4.741606e+13,42767.080,10.591,1.433245e+05
2114785,충북,휴양콘도 운영업,제주,50s,1,4,7,2020,0,3,8.806154,False,4.741606e+13,42767.080,10.591,1.433245e+05
2114786,충북,휴양콘도 운영업,제주,50s,1,4,7,2020,0,0,11.461490,False,4.741606e+13,42767.080,10.591,NaN


# dacon의 temp merge with **data** : temp_data.csv로 저장

In [22]:
df_num_drop = df_num.drop(['Unnamed: 0','year','month','AMT'],axis = 1)

In [23]:
df_num_drop

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,deceased,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby
0,강원,건강보조식품 소매업,강원,20s,1,1,0,0,8.806154,False,1.562276e+13,42767.077,10.591,1.980289e+06
1,강원,건강보조식품 소매업,강원,20s,1,1,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
2,강원,건강보조식품 소매업,강원,20s,1,1,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
3,강원,건강보조식품 소매업,강원,20s,1,1,0,11,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
4,강원,건강보조식품 소매업,강원,20s,1,1,1,0,7.066787,True,1.562276e+13,42767.077,10.591,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057389,충북,휴양콘도 운영업,충북,70s,2,5,0,0,7.066787,False,4.267602e+11,42767.080,10.591,NaN
1057390,충북,휴양콘도 운영업,충북,70s,2,5,0,0,8.806154,False,4.267602e+11,42767.080,10.591,1.433245e+05
1057391,충북,휴양콘도 운영업,충북,70s,2,5,0,0,7.066787,False,4.267602e+11,42767.080,10.591,1.433245e+05
1057392,충북,휴양콘도 운영업,충북,70s,2,5,0,0,7.066787,False,4.267602e+11,42767.080,10.591,1.433245e+05


In [24]:
df_num_drop.shape

(1057394, 14)

In [25]:
df_num_drop.dtypes

CARD_SIDO_NM                    object
STD_CLSS_NM                     object
HOM_SIDO_NM                     object
AGE                             object
SEX_CTGO_CD                      int64
FLC                              int64
deceased                         int64
festival_count                   int64
rain_day                       float64
covid                             bool
age_population                 float64
total_sale_by_indst_groupby    float64
log_total_sale_by_indst        float64
total_tourist_groupby          float64
dtype: object

In [26]:
# ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat
temp.dtypes
int_features = ['SEX_CTGO_CD','FLC']
for i in enumerate (int_features) : 
    ca = i[1] 
    temp[ca] = temp[ca].astype('int64') 

In [27]:
temp_data = pd.merge(temp, train_num_drop, on=['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE', 'SEX_CTGO_CD', 'FLC'])

In [28]:
temp_data.shape

(2114788, 16)

In [29]:
temp_data

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,deceased,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby
0,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,0,0,5.297286,False,1.562276e+13,42767.077,10.591,NaN
1,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,0,4,5.297286,False,1.562276e+13,42767.077,10.591,1.980289e+06
2,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,0,4,5.297286,False,1.562276e+13,42767.077,10.591,1.980289e+06
3,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,0,0,8.806154,False,1.562276e+13,42767.077,10.591,1.980289e+06
4,강원,건강보조식품 소매업,강원,20s,1,1,4,2020,0,8,11.461487,False,1.562276e+13,42767.077,10.591,1.980289e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114783,충북,휴양콘도 운영업,제주,50s,1,4,4,2020,0,0,11.461490,False,4.741606e+13,42767.080,10.591,NaN
2114784,충북,휴양콘도 운영업,제주,50s,1,4,4,2020,0,2,8.806154,False,4.741606e+13,42767.080,10.591,1.433245e+05
2114785,충북,휴양콘도 운영업,제주,50s,1,4,7,2020,0,3,8.806154,False,4.741606e+13,42767.080,10.591,1.433245e+05
2114786,충북,휴양콘도 운영업,제주,50s,1,4,7,2020,0,0,11.461490,False,4.741606e+13,42767.080,10.591,NaN


------------------------------------------------------------------------------------

In [30]:
temp_data == temp_train_num

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,year,month,deceased,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114783,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False
2114784,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2114785,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2114786,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,False


In [31]:
np.count_nonzero(temp_data == temp_train_num)

33225606

In [32]:
np.sum(temp_data == temp_train_num)

CARD_SIDO_NM                   2114788
STD_CLSS_NM                    2114788
HOM_SIDO_NM                    2114788
AGE                            2114788
SEX_CTGO_CD                    2114788
FLC                            2114788
year                           2114788
month                          2114788
deceased                       2114788
festival_count                 2114788
rain_day                       2114788
covid                          2114788
age_population                 2114788
total_sale_by_indst_groupby    2080032
log_total_sale_by_indst        2080032
total_tourist_groupby          1573298
dtype: int64

아래 세 줄에서 같지 않은 것이 무더기로 나왔다
total_sale_by_indst_groupby, log_total_sale_by_indst, total_tourist_groupby 빼고는 전체 값 동일  

-----------------------------------------------------------

# [2] lightGBM baseline code
## 주의 : [홈페이지 베이스라인 코드](https://dacon.io/competitions/official/235615/codeshare/1322?page=1&dtype=recent&ptype=pub)상 인코딩, 디코딩을 건너뛰고 수작업으로 하였으므로, 나머지 팀원 분들은 인코딩 디코딩을 쓰는 것을 추천합니다..(헷갈려서 시간을 많이 썼어요.) /  맨 아래에 홈페이지 상 베이스라인 코드도 넣어놨어요.

In [33]:
x = train_num.drop(['AMT','Unnamed: 0'], axis=1)
y = np.log1p(train_num['AMT'])

In [34]:
x.head(3)

,CARD_SIDO_NM,STD_CLSS_NM,HOM_SIDO_NM,AGE,SEX_CTGO_CD,FLC,month,year,deceased,festival_count,rain_day,covid,age_population,total_sale_by_indst_groupby,log_total_sale_by_indst,total_tourist_groupby
149245,경남,그외 기타 분류안된 오락관련 서비스업,경남,30s,1,2,3,2019,0,5,8.806154,False,1.078704e+14,42767.08,10.591,304026.9
554956,서울,일반유흥 주점업,인천,20s,1,2,12,2019,0,1,8.806154,False,5.395069e+14,721586.00,13.470,9279775.0
918516,제주,차량용 주유소 운영업,제주,50s,2,4,5,2019,0,17,11.461490,False,4.596082e+13,721586.00,13.470,1980289.0


In [35]:
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM','AGE','covid']
for i in enumerate (cat_features) : 
    ca = i[1] 
    x[ca] = x[ca].astype('category') 

In [36]:
k = int(len(x)*0.9)

In [37]:
#from sklearn.impute import KNNImputer

In [38]:
x_train = x[:k]
y_train = y[:k]
x_val = x[k:]
y_val = y[k:]

In [39]:
x_train.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_train.columns]
x_val.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in x_val.columns]

In [40]:
import lightgbm as lgb

In [41]:
train_ds = lgb.Dataset(x_train, label=y_train)
val_ds = lgb.Dataset(x_val, label=y_val)

In [42]:
params = {
            'learning_rate' : 0.05,
            'boosting_type': 'gbdt',
            'objective': 'tweedie',
            'tweedie_variance_power': 1.1,
            'metric': 'rmse',
            'sub_row' : 0.75,
            'lambda_l2' : 0.1
        }

In [43]:
model = lgb.train(params,
                  train_ds,
                  1000,
                  val_ds,
                  verbose_eval = 100,
                  early_stopping_rounds = 100
                 )

Training until validation scores don't improve for 100 rounds
[100]	valid_0's rmse: 1.75004
[200]	valid_0's rmse: 1.46256
[300]	valid_0's rmse: 1.28882
[400]	valid_0's rmse: 1.18831
[500]	valid_0's rmse: 1.11465
[600]	valid_0's rmse: 1.04539
[700]	valid_0's rmse: 0.996176
[800]	valid_0's rmse: 0.953523
[900]	valid_0's rmse: 0.923172
[1000]	valid_0's rmse: 0.903073
Did not meet early stopping. Best iteration is:
[1000]	valid_0's rmse: 0.903073


In [44]:
temp_data.dtypes
obj_features = ['CARD_SIDO_NM', 'STD_CLSS_NM', 'HOM_SIDO_NM', 'AGE','covid']
for i in enumerate (obj_features) : 
    ca = i[1] 
    temp_data[ca] = temp_data[ca].astype('category') 

obj_features = ['year', 'month']
for i in enumerate (obj_features) : 
    ca = i[1] 
    temp_data[ca] = temp_data[ca].astype('int64') 

In [45]:
temp_data.dtypes

CARD_SIDO_NM                   category
STD_CLSS_NM                    category
HOM_SIDO_NM                    category
AGE                            category
SEX_CTGO_CD                       int64
FLC                               int64
year                              int64
month                             int64
deceased                          int64
festival_count                    int64
rain_day                        float64
covid                          category
age_population                  float64
total_sale_by_indst_groupby     float64
log_total_sale_by_indst         float64
total_tourist_groupby           float64
dtype: object

In [46]:
x_val.dtypes

CARD_SIDO_NM                   category
STD_CLSS_NM                    category
HOM_SIDO_NM                    category
AGE                            category
SEX_CTGO_CD                       int64
FLC                               int64
month                             int64
year                              int64
deceased                          int64
festival_count                    int64
rain_day                        float64
covid                          category
age_population                  float64
total_sale_by_indst_groupby     float64
log_total_sale_by_indst         float64
total_tourist_groupby           float64
dtype: object

In [47]:
x_train.dtypes

CARD_SIDO_NM                   category
STD_CLSS_NM                    category
HOM_SIDO_NM                    category
AGE                            category
SEX_CTGO_CD                       int64
FLC                               int64
month                             int64
year                              int64
deceased                          int64
festival_count                    int64
rain_day                        float64
covid                          category
age_population                  float64
total_sale_by_indst_groupby     float64
log_total_sale_by_indst         float64
total_tourist_groupby           float64
dtype: object

In [48]:
#train(x_train) and valid dataset(temp_data) categorical_feature do not match. 확인완료

In [49]:
# 예측
pred = model.predict(temp_data)
pred = np.expm1(pred)
temp_data['AMT'] = np.round(pred, 0)
temp_data['REG_YYMM'] = temp_data['year']+ temp_data['month']*100
temp_data = temp_data[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
temp_data = temp_data.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False)

In [50]:
#디코딩?
cat_features = ['CARD_SIDO_NM', 'STD_CLSS_NM']
for i in enumerate (cat_features) : 
    ca = i[1] 
    temp_data[ca] = temp_data[ca].astype('object') 

In [51]:
temp_data.head(3)

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
0,202004,강원,건강보조식품 소매업,8.783928e+08
1,202004,강원,골프장 운영업,1.915112e+10
2,202004,강원,과실 및 채소 소매업,7.529156e+09


In [52]:
# 제출 파일 만들기
submission = pd.read_csv('C:/Users/USER/Desktop/[2020]/2020 summer/데이콘/jeju_data_ver1/submission.csv', index_col=0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(temp_data, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission.csv', encoding='utf-8-sig')
submission.head()

,REG_YYMM,CARD_SIDO_NM,STD_CLSS_NM,AMT
id,,,,
0,202004,강원,건강보조식품 소매업,8.783928e+08
1,202004,강원,골프장 운영업,1.915112e+10
2,202004,강원,과실 및 채소 소매업,7.529156e+09
3,202004,강원,관광 민예품 및 선물용품 소매업,2.062371e+08
4,202004,강원,그외 기타 분류안된 오락관련 서비스업,1.679100e+05


------------------------------------------------------------------

# [3] 홈페이지 상 베이스라인 코드

In [ ]:
# 인코딩
dtypes = df.dtypes
encoders = {}
for column in df.columns:
    if str(dtypes[column]) == 'object':
        encoder = LabelEncoder()
        encoder.fit(df[column])
        encoders[column] = encoder
        
df_num = df.copy()        
for column in encoders.keys():
    encoder = encoders[column]
    df_num[column] = encoder.transform(df[column])

In [260]:
# 디코딩 
temp['CARD_SIDO_NM'] = encoders['CARD_SIDO_NM'].inverse_transform(temp['CARD_SIDO_NM'])
temp['STD_CLSS_NM'] = encoders['STD_CLSS_NM'].inverse_transform(temp['STD_CLSS_NM'])

NameError: name 'encoders' is not defined